In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("Streaming_Temperature_Alerts") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()

In [2]:
schema = StructType([
    StructField("sensor_id", StringType(), True),
    StructField("temperature", DoubleType(), True),
    StructField("timestamp", StringType(), True)
])

In [3]:
df_temp = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "temperature-sensors") \
    .option("startingOffsets", "earliest") \
    .load()

In [4]:
temp_logic = df_temp.select(
    from_json(col("value").cast("string"), schema).alias("data")
).select("data.*") \
.withColumn("ALERTA", col("temperature") > 25.0)

In [5]:
query_temp = temp_logic.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("tabla_temp_live") \
    .start()

In [16]:
spark.sql("select * from tabla_temp_live order by timestamp desc").show()

+-------------+-----------+--------------------+------+
|    sensor_id|temperature|           timestamp|ALERTA|
+-------------+-----------+--------------------+------+
|sensor_temp_2|       29.0|2026-02-20T17:10:...|  true|
|sensor_temp_1|      25.05|2026-02-20T17:10:...|  true|
|sensor_temp_2|      29.34|2026-02-20T17:10:...|  true|
|sensor_temp_1|      25.08|2026-02-20T17:10:...|  true|
|sensor_temp_2|      29.44|2026-02-20T17:10:...|  true|
|sensor_temp_1|      24.85|2026-02-20T17:10:...| false|
|sensor_temp_2|      29.03|2026-02-20T17:10:...|  true|
|sensor_temp_1|      24.93|2026-02-20T17:10:...| false|
|sensor_temp_2|      29.33|2026-02-20T17:10:...|  true|
|sensor_temp_1|      24.92|2026-02-20T17:09:...| false|
|sensor_temp_2|      29.26|2026-02-20T17:09:...|  true|
|sensor_temp_1|       25.1|2026-02-20T17:09:...|  true|
|sensor_temp_2|      29.06|2026-02-20T17:09:...|  true|
|sensor_temp_1|      25.18|2026-02-20T17:09:...|  true|
|sensor_temp_2|      29.23|2026-02-20T17:09:...|